In [2]:
"""
块是由单个层或多个层所组合而成的组件或模型本身
块可以和块进行组合,从而形成更大的组件(这一过程通常是递归的)
从编程的角度,块由class表示。它的任何子类都必须定义一个将其输入转换为输出的前向传播函
数,并且必须存储任何必需的参数。

自定义块需要实现的基本功能：
1. 将输入数据作为其前向传播函数的参数。
2. 通过前向传播函数来生成输出。(输出的形状可能与输入的形状不同)
3. 计算其输出关于输入的梯度,可通过其反向传播函数进行访问。通常这是自动发生的。
4. 存储和访问前向传播计算所需的参数。
5. 根据需要初始化模型参数。
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
class MLP(nn.Module): # 自定义一个MLP块,这个块继承自Pytorch中表示块的类Module
    def __init__(self) -> None:
        super().__init__()
        self.hidden = nn.Linear(20,256) # 隐藏层
        self.out = nn.Linear(256,10) # 输出层

    def forward(self, X): # 定义模型的前向传播,及根据输入X返回所需的模型输出
        return self.out(F.relu(self.hidden(X)))
X = torch.rand(2,20)
net = MLP()
net(X)

class MySequential(nn.Module): # 自定义一个顺序块(nn.Sequential)
    """
    顺序块是为了把其他模块串起来,即需要定义两个函数：
    1. 一种将块逐个追加到列表中的函数；
    2. 一种前向传播函数,用于将输入按追加块的顺序传递给块组成的“链条”。
    """
    def __init__(self, *args) -> None:
        super().__init__()
        for idx, module in enumerate(args):
            """
            这里module是Module子类的一个实例(就如同在nn.Sequtial中填入的nn.Linear)。
            其被保存在'Module'类的成员变量_modules中。_module的类型是OrderedDict
            使用_module属性而不是自定义python列表的好处是
            在模块的参数初始化过程中,系统知道在_modules字典中查找需要初始化参数的子块
            """
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

tensor([[-0.1382,  0.0983, -0.0709,  0.0051,  0.0983, -0.0698, -0.0101,  0.3989,
         -0.1966, -0.0110],
        [ 0.0070,  0.2035,  0.0131,  0.1082,  0.0804, -0.0274,  0.0054,  0.4127,
         -0.0974, -0.0523]], grad_fn=<AddmmBackward0>)